In [ ]:
# Brain Tumor Recognition

# Vorgehen
1. Data preprocessing
2. Convulutional NN aufbauen
3. CNN trainieren
4. Analysieren
5. Falls möglich: Dense Network anhängen

# Data Preprocessing

In [2]:
%pip install tensorflow_datasets

  Using cached tensorflow_datasets-4.9.7-py3-none-any.whl.metadata (9.6 kB)
  Using cached dm_tree-0.1.8-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
  Using cached promise-2.3-py3-none-any.whl
  Using cached simple_parsing-0.1.6-py3-none-any.whl.metadata (7.3 kB)
  Using cached tensorflow_metadata-1.16.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached array_record-0.5.1-1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (699 bytes)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
Using cached tensorflow_datasets-4.9.7-py3-none-any.whl (5.3 MB)
Using cached array_record-0.5.1-1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (2.2 MB)
Using cached dm_tree-0.1.8-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (152 kB)
Using cached simple_parsing-0.1.6-py3-none-any.whl (112 kB)
Using cached tensorflow_metadata-1.16.1-py3-none-a

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [4]:
import pandas as pd

In [10]:
import os
import tensorflow as tf

# Basisordner für die Daten (Korrigierter Pfad)
base_dir = r"C:/Users/leoni/Documents/HSLU/5. Semester/AI Einzelarbeit/Daten"  # Rohstring für Windows-Pfad
train_dir = os.path.join(base_dir, "Training")
test_dir = os.path.join(base_dir, "Testing")

# Parameter für die Datenverarbeitung
batch_size = 32
img_height = 224
img_width = 224

# Training-Datensatz laden
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,  # Optional: Training und Validierung aufteilen
    subset="training",
    seed=123,
    image_size=(img_height, img_width),  # Bilder auf einheitliche Größe skalieren
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,  # Optional: Validierungsanteil
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Test-Datensatz laden
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Überprüfung der geladenen Daten
print("Trainingsdatenklassen:", train_ds.class_names)
print("Validierungsdatenklassen:", val_ds.class_names)
print("Testdatenklassen:", test_ds.class_names)


NotFoundError: Could not find directory C:/Users/leoni/Documents/HSLU/5. Semester/AI Einzelarbeit/Daten/Training

In [19]:
print("Existiert der Trainingsordner?", os.path.exists(r"C:\Users\leoni\Documents\HSLU"))

Existiert der Trainingsordner? False
